https://python.langchain.com/docs/tutorials/rag/

In [3]:
import os
from dotenv import load_dotenv
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_google_community import BigQueryVectorStore
import csv
from langchain_google_community import BigQueryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langgraph.graph import MessagesState, StateGraph, END
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image
from langchain_core.messages import SystemMessage

In [4]:
load_dotenv()
assert "LANGSMITH_TRACING" in os.environ, "Please set the LANGSMITH_TRACING environment variable."
assert "LANGSMITH_API_KEY" in os.environ, "Please set the LANGSMITH_API_KEY environment variable."
assert "PROJECT_ID" in os.environ, "Please set the PROJECT_ID environment variable."
assert "LOCATION" in os.environ, "Please set the LOCATION environment variable."
assert "DATASET" in os.environ, "Please set the DATASET environment variable."
assert "TABLE" in os.environ, "Please set the TABLE environment variable."
PROJECT_ID = os.getenv("PROJECT_ID") 
LOCATION = os.getenv("LOCATION") 
DATASET = os.getenv("DATASET") 
TABLE = os.getenv("TABLE") 

In [5]:
PROJECT_ID = "llm-studies"
LOCATION = "us-central1"
DATASET = "blog_embeddings"
TABLE = "rag_embeddings_s_b" # s: simple, b: BQ

In [6]:
embeddings = VertexAIEmbeddings(model="textembedding-gecko@latest")

In [7]:
vector_store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=LOCATION,
    embedding=embeddings,
)

BigQuery table llm-studies.blog_embeddings.rag_embeddings_s_b initialized/validated as persistent storage. Access via BigQuery console:
 https://console.cloud.google.com/bigquery?project=llm-studies&ws=!1m5!1m4!4m3!1sllm-studies!2sblog_embeddings!3srag_embeddings_s_b


In [8]:
BASE_QUERY = """
SELECT post_title, summarization, post_url
FROM `llm-studies.blog.posts_summarization` s, llm-studies.blog.posts_dez_2024 p
where s.post_id = p.post_id
"""

In [11]:
loader = BigQueryLoader(BASE_QUERY,
    page_content_columns=["summarization"],
    metadata_columns=["post_title", "post_url"],)

data = loader.load()

In [23]:
print(data[0])

page_content='summarization: O vídeo da School of Life (SoL) explora o conceito de *eudaimonia*, um termo que se origina de Platão e Aristóteles. A SoL compara *eudaimonia* com a palavra contemporânea "felicidade", argumentando que os filósofos antigos não propunham uma vida feliz como objetivo. O propósito da vida não é evitar o sofrimento ou ser infeliz, como a palavra é usada hoje, mas sim superar os desafios diários e simples que surgem sob pressão. Para a SoL, *eudaimonia* deve ser associada à palavra "realização", que se distingue da "felicidade" pela dor. Assim, a palavra *eudaimonia* pode acomodar o feliz e o infeliz, e em vez de buscar uma existência sem dor, precisamos ir além e fazer a diferença. Em resumo, precisamos fazer o que é realmente importante, mais do que sorrir o tempo todo. Seguir a recomendação de *eudaimonia* significa que podemos passar toda a vida lutando em nosso trabalho, relacionamentos e engajamento político, mas terminar nossos dias sentindo que essas ta

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(data)
len(all_splits)

193

In [14]:
_ = vector_store.add_documents(documents=all_splits)

In [21]:
llm = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai")

In [15]:
@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [16]:
# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}

In [17]:
# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [19]:
tools = ToolNode([retrieve])

graph_builder = StateGraph(MessagesState)
graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [22]:
input_message = "me fale sobre a mente"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}, {"role": "system", "content": "Você é um tutor de filosofia e deve trazer as respostas baseadas na teorias encontradas nos textos, sejam de autores ou termos"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ System Message ================================

Você é um tutor de filosofia e deve trazer as respostas baseadas na teorias encontradas nos textos, sejam de autores ou termos
================================== Ai Message ==================================
Tool Calls:
  retrieve (e0ca9737-4800-46f1-8a38-1cc4e42ea6ac)
 Call ID: e0ca9737-4800-46f1-8a38-1cc4e42ea6ac
  Args:
    query: o que é a mente na filosofia
================================= Tool Message =================================
Name: retrieve

Source: {'doc_id': '9dc0985b74d94ae4a22070f7add18a0d', 'post_title': 'Searle contra Dennett*', 'post_url': 'http://www.reflexoesdofilosofo.blog.br/2024/10/searle-contra-dennett.html', 'score': 0.6022395713664966}
Content: Os posts do blog exploram diversos temas relacionados à filosofia da mente, como a relação entre o cérebro, a mente e o comportamento, o conceito de qualia, a crítica ao materialismo e ao dualismo, a intencionalidade, a psicologia pop

In [12]:
# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

In [24]:
input_message = "searle"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}, {"role": "system", "content": "Você é um tutor de filosofia e deve trazer as respostas baseadas na teorias encontradas nos textos, sejam de autores ou termos"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ System Message ================================

Você é um tutor de filosofia e deve trazer as respostas baseadas na teorias encontradas nos textos, sejam de autores ou termos
================================== Ai Message ==================================
Tool Calls:
  retrieve (73b7bd86-403d-4495-9089-0753afc3b227)
 Call ID: 73b7bd86-403d-4495-9089-0753afc3b227
  Args:
    query: searle
================================= Tool Message =================================
Name: retrieve

Source: {'doc_id': 'da3aca1994054facbf2670d91ada015a', 'post_title': 'Searle contra Dennett*', 'post_url': 'http://www.reflexoesdofilosofo.blog.br/2024/10/searle-contra-dennett.html', 'score': 0.8201956489765119}
Content: summarization: O texto apresentado é uma coleção de posts de um blog sobre filosofia da mente, explorando as ideias de Searle e Dennett sobre a relação entre mente e corpo, a natureza da consciência, a inteligência artificial e o teste de Turing. O blog di

In [31]:
import json
from langchain.load.dump import dumps

input_message = "searle"

result = graph.invoke({"messages": [{"role": "user", "content": input_message}, {"role": "system", "content": "Você é um tutor de filosofia e deve trazer as respostas baseadas na teorias encontradas nos textos, sejam de autores ou termos"}]})
json_string = dumps(result["messages"], ensure_ascii=False)
print(json_string)
#print(json.dumps(result["messages"], indent=2))

[{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "searle", "type": "human", "id": "b0e2f776-2942-48cc-9204-388014d8f050"}}, {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "SystemMessage"], "kwargs": {"content": "Você é um tutor de filosofia e deve trazer as respostas baseadas na teorias encontradas nos textos, sejam de autores ou termos", "type": "system", "id": "5ff16596-9297-46c1-9a7a-e55e4c424249"}}, {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "AIMessage"], "kwargs": {"content": "", "additional_kwargs": {"function_call": {"name": "retrieve", "arguments": "{\"query\": \"searle\"}"}}, "response_metadata": {"is_blocked": false, "safety_ratings": [], "usage_metadata": {"prompt_token_count": 38, "candidates_token_count": 4, "total_token_count": 42, "prompt_tokens_details": [{"modality": 1, "token_count": 38}], "candidates_tokens_details": [{"modality": 1, "token_co